In [1]:
from publisher.logging.logger import logger, session_id


In [2]:
from time import sleep

from config import Config

import publisher.data_source_gateways.collector.collector as collector_gateway
from publisher.publisher_service import publish_news

config = Config()


In [7]:
keywords_response = collector_gateway.get_keywords(
    sources=["tagesschau"],
    url=config.collector["url"],
    endpoint=config.collector["endpoints"]["keywords"],
    session_id=config.session_id
)


In [22]:
import feedparser

rss_feed_url = 'https://www.calnewport.com/blog/feed/'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Print the titles of the latest 5 blog posts
for post in feed.entries[:5]:
    print(post.title)


Meta Rediscovers the Cubicle
On Section 230 and the Dream of a More Human Internet
Pliny the Younger on Happy and Honorable Seclusion
On Email and Horses
Guillermo del Toro’s Inspiration Machine


In [23]:
some_set = set()

for i in (i.keys() for i in feed.entries):
    some_set.update(i)

some_set


{'author',
 'author_detail',
 'authors',
 'comments',
 'content',
 'guidislink',
 'id',
 'link',
 'links',
 'published',
 'published_parsed',
 'slash_comments',
 'summary',
 'summary_detail',
 'tags',
 'title',
 'title_detail',
 'wfw_commentrss'}

In [5]:
post

{'title': 'Meta Rediscovers the Cubicle',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://calnewport.com/blog/feed/',
  'value': 'Meta Rediscovers the Cubicle'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://calnewport.com/meta-rediscovers-the-cubicle/'}],
 'link': 'https://calnewport.com/meta-rediscovers-the-cubicle/',
 'comments': 'https://calnewport.com/meta-rediscovers-the-cubicle/#comments',
 'authors': [{'name': 'Study Hacks'}],
 'author': 'Study Hacks',
 'author_detail': {'name': 'Study Hacks'},
 'published': 'Thu, 09 Mar 2023 21:54:35 +0000',
 'published_parsed': time.struct_time(tm_year=2023, tm_mon=3, tm_mday=9, tm_hour=21, tm_min=54, tm_sec=35, tm_wday=3, tm_yday=68, tm_isdst=0),
 'tags': [{'term': 'Uncategorized', 'scheme': None, 'label': None}],
 'id': 'https://calnewport.com/?p=15270',
 'guidislink': False,
 'summary': '<p>Back in 2016, I reported on a rumor that was circulating about employee dissatisfaction at

In [15]:
import feedparser

rss_feed_url = 'https://www.omnisophie.com/feed/'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Print the titles and links of the latest 10 blog posts
for post in feed.entries[:15]:
    print(post.title)
    print(post.link)
    print()


Psychothese: Deutschland als vorsichtiger Versager und planloser Rater
https://www.omnisophie.com/psychothese-deutschland-als-vorsichtiger-versager-und-planloser-rater/

Hass-Magneten
https://www.omnisophie.com/hass-magneten/

Handlungs-Chaos auf der Bühne – wie es uns gefällt
https://www.omnisophie.com/handlungs-chaos-auf-der-buehne-wie-es-uns-gefaellt/

BWLer verwechseln Reserven mit Verschwendung – wir auch
https://www.omnisophie.com/bwler-verwechseln-reserven-mit-verschwendung-wir-auch/

Wahr ist nur, was ohne Interesse richtig aussieht
https://www.omnisophie.com/wahr-ist-nur-was-ohne-interesse-richtig-aussieht/

Wenn Probleme unbemerkt exponentiell wachsen
https://www.omnisophie.com/wenn-probleme-unbemerkt-exponentiell-wachsen/

Homöophorie – eine Höherentwicklung der Homöopathie
https://www.omnisophie.com/homoeophorie-eine-hoeherentwicklung-der-homoeopathie/

Der Dreiklang der Wertschwätzung zu einem unkonkreten Wahlprogramm
https://www.omnisophie.com/der-dreiklang-der-wertschwae

In [21]:
some_set = set()

for i in (i.keys() for i in feed.entries):
    some_set.update(i)

some_set


{'author',
 'author_detail',
 'authors',
 'comments',
 'content',
 'guidislink',
 'id',
 'link',
 'links',
 'published',
 'published_parsed',
 'slash_comments',
 'summary',
 'summary_detail',
 'tags',
 'title',
 'title_detail',
 'wfw_commentrss'}

In [48]:
news_1 = [i for i in range(50)]
news_2 = [i for i in range(50, 100)]
news_3 = [i for i in range(100, 150)]

news = [
    news_1,
    news_2,
    news_3,
]

def get_news():
    news_response = []

    for i in news:
        yield news_response.extend(i)

    return news_response


In [49]:
response = get_news()


In [52]:
import itertools

all_gen = [(i for i in range(5)), (i for i in range(5, 10))]

new_gen = itertools.chain(*all_gen)

# Print the values generated by the new generator
for value in new_gen:
    print(value)


0
1
2
3
4
5
6
7
8
9


In [54]:
import requests


In [55]:
url = "https://www.tagesschau.de/api2"
endpoint = "/news"


In [56]:
response = requests.get(f"{url}{endpoint}")


In [58]:
json_response = response.json()


In [59]:
json_response["news"][0]


{'sophoraId': 'bankenkrise-ansteckungsgefahr-101',
 'externalId': 'b2050486-3a96-4b4b-9e44-e22d6e8b25d2',
 'title': 'Wie hoch ist die Ansteckungsgefahr?',
 'teaserImage': {'title': '',
  'copyright': 'picture alliance/dpa',
  'alttext': 'Frankfurter Hochhäuser und Bankentürme',
  'type': 'image',
  'videowebl': {'imageurl': 'https://www.tagesschau.de/multimedia/bilder/frankfurt-315~_v-videowebl.jpg'},
  'klein1x1': {'imageurl': 'https://www.tagesschau.de/multimedia/bilder/frankfurt-315~_v-klein1x1.jpg'},
  'portraetgrossplus8x9': {'imageurl': 'https://www.tagesschau.de/multimedia/bilder/frankfurt-315~_v-portraetgrossplus8x9.jpg'},
  'videoweb1x1l': {'imageurl': 'https://www.tagesschau.de/multimedia/bilder/frankfurt-315~_v-videoweb1x1l.jpg'},
  'videowebm': {'imageurl': 'https://www.tagesschau.de/multimedia/bilder/frankfurt-315~_v-videowebm.jpg'},
  'videowebs': {'imageurl': 'https://www.tagesschau.de/multimedia/bilder/frankfurt-315~_v-videowebs.jpg'},
  'portraetgross8x9': {'imageurl':

In [5]:
from typing import Union

from pydantic import BaseModel


class TopicMappingHelper(BaseModel):
    germany: Union[str, None] = None
    world: Union[str, None] = None
    business: Union[str, None] = None
    sports: Union[str, None] = None
    technology: Union[str, None] = None
    entertainment: Union[str, None] = None
    science: Union[str, None] = None
    health: Union[str, None] = None


topic_mapping = {
    "germany": "inland",
    "world": "ausland",
    "business": "wirtschaft",
    "sports": "sport",
    # "video",
    # "investigativ",
    # "faktenfinder",
    "technology": None,
    "entertainment": None,
    "science": None,
    "health": None,
}


In [2]:
dict(zip(topic_mapping.values(), topic_mapping.keys()))


{'inland': 'Germany',
 'ausland': 'World',
 'wirtschaft': 'Business',
 'sport': 'Sports',
 None: 'Health'}

In [10]:
topic_mapping_helper = TopicMapping(
    germany="inland",
    world="ausland",
    business="wirtschaft",
    sports="sport",
).dict()

topic_mapping = dict(zip(topic_mapping_helper.values(), topic_mapping_helper.keys()))
del topic_mapping[None]


In [11]:
topic_mapping


{'inland': 'germany',
 'ausland': 'world',
 'wirtschaft': 'business',
 'sport': 'sports'}